# Planning-Lab Lesson 1: Search Strategies

In this first lesson we will work both on Search Strategies. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [10]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs

# Uninformed search
### Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**Here's an implementation of the BFS algorithm in its Tree Search version:**

In [11]:
def BFS_TreeSearch(problem):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    
    if node.state == problem.goalstate:
        return build_path(node), time_cost, space_cost
    
    frontier = NodeQueue()
    frontier.add(node)    
    
    while not frontier.is_empty():
        node = frontier.remove()
        
        for action in range(problem.action_space.n):
            
            child = Node(problem.sample(node.state, action), node)
            time_cost += 1
            
            if problem.goalstate == child.state: 
                return build_path(child), time_cost, space_cost
            
            frontier.add(child)
             
        space_cost = max(space_cost, len(frontier))  

    return build_path(node), time_cost, space_cost

**Now you can implement the same BFS algorithm in the Graph Search version:**

In [12]:
def BFS_GraphSearch(problem):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    #
    # YOUR CODE HERE ...
    #
    return build_path(node), time_cost, space_cost     


**The following code calls your graph search version and the tree search version of BFS and checks the results:**

In [13]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Your solution: []
N° of nodes explored: 1
Max n° of nodes in memory: 1

> The solution is not correct, should be: 
[(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:

<img src="images/dls.png" width="600">

<img src="images/ids.png" width="600">

Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [14]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default
child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [15]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

In [16]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    if problem.goalstate == node.state: # Goal state check
        return build_path(node), 1, node.depthcost
    elif limit == 0: # Limit budget check for cutoff
        return "cut_off", 1, node.depthcost
    
    space_cost = node.depthcost
    time_cost = 1 
    
    cut_off_occurred = False
    for action in range(problem.action_space.n): # Look around
        child = Node(problem.sample(node.state, action), node) # Child node 
        result = Recursive_DLS_TreeSearch(child, problem, limit-1) # Recursive call
        time_cost += result[1]

        space_cost = max(space_cost, result[2])
        
        if result[0] == "cut_off":
            cut_off_occurred = True
        elif result[0] != "failure": # Solution found
            return result[0], time_cost, space_cost
        
    if(cut_off_occurred): # Solution not found but cutoff occurred
        return "cut_off", time_cost, space_cost
    else: # No solution and no cutoff: failure
        return "failure", time_cost, space_cost

**Study the provided code for Depth Limited Search (DLS) and Recursive_DLS_TreeSearch and implement the Iterative Deepening Search function reported below.**

In [17]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
  
    total_time_cost = 0
    total_space_cost = 1
    for i in zero_to_infinity():
        #
        # YOUR CODE HERE ...
        #
        return 0, 0, 0, 0 # Placeholder
    return solution_dls, total_time_cost, total_space_cost, i


The following code calls your version of IDS and checks the results:

In [18]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], environment)
results.check_sol_ts()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 0
Your solution: 0
N° of nodes explored: 0
Max n° of nodes in memory: 0

> Your necessary iterations are not correct, should be: 9



#### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?